CNN med Pytorch

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import sys
from pathlib import Path

#torch 
import torch
import torchvision as torchv
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch import Tensor
from torch.utils import data

from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sn

In [2]:
module_path = str(Path.cwd().parents[0].parents[0] / "methods")

if module_path not in sys.path:
    sys.path.append(module_path)

from dataloader import *
# from plotCreator import *

data_path0 = str(Path.cwd().parents[0].parents[0] / "data" / "BH_n4_M10_res50_15000_events.h5")
data_path1 = str(Path.cwd().parents[0].parents[0] / "data" / "PP13-Sphaleron-THR9-FRZ15-NB0-NSUBPALL_res50_15000_events.h5")

In [3]:
bhArray = dataToArray(data_path0)
sphArray = dataToArray(data_path1)

# print(bhArray.shape)
# print(sphArray.shape)

In [4]:
#Kombinerer dataene for å kunne kjøre gjennom modellen på et samlet datasett
dataArray = np.concatenate((bhArray,sphArray),axis=0)

# np.shape(dataArray)

In [5]:
# Labeler tabelle med 1 og 0 (0 = svart hull, 1 = spahleron)
labelsArray = np.concatenate((np.zeros(np.shape(bhArray)[0]),np.ones(np.shape(sphArray)[0])),axis=0)

Sjekk om køyrer på GPU eller CPU

In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [7]:
#Split data 75% i train og 25% i test
trainData, testData, trainLabels, testLabels = train_test_split(dataArray, labelsArray, random_state=42)

In [8]:
trainData = torch.from_numpy(trainData)
testData = torch.from_numpy(testData)
trainLabels = torch.from_numpy(trainLabels)
testLabels = torch.from_numpy(testLabels)

In [9]:
#Transformere numpy til tensor
train = torch.utils.data.TensorDataset(trainData, trainLabels)
test = torch.utils.data.TensorDataset(testData, testLabels)

In [10]:
#Sette data til å vere dataloaders
trainLoader = DataLoader(train, shuffle=True, batch_size=50)
testLoader = DataLoader(test, shuffle=True, batch_size=50)

Her er vårt CNN modell

Her skal det komme endrigane for å tilpasse første CNN

In [15]:
# Her kommer modellen inn
class ConvModel(nn.Module):
    #Husk art her brukes droput for å unngå overfitting
    def __init__(self, dropout):

        super(ConvModel, self).__init__()
        #opprette conv lag. Bildene har 3 lag 
        
        #Dette funker
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=0)
        
        #Her prøver å tweake på første pooling laget
        # Define the convolutional layers with rotational and reflectional symmetry filters
        # self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=0)
        # self.conv1_filter = nn.Parameter(torch.ones((16, 3, 3, 3)))
        # self.conv1_filter.detach_()
        
        # self.conv1_filter[:, :, 1, 1] = 0  # set the center of the filter to zero
        # self.conv1_filter[:, :, :, 1] = self.conv1_filter.flip(dims=[3])  # set the right half of the filter
        # self.conv1_filter[:, :, 0, 0] = self.conv1_filter[:, :, 2, 2]  # set the top-left and bottom-right corners
        
        
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=64, kernel_size=3, padding=0)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, padding=0)
        #kanskje ha flere layers?

        #Unchange
        self.fc1 = nn.Linear(3*3*256, 128)
        self.fc2 = nn.Linear(128,2)

        self.dropout = nn.Dropout(dropout)


    #Enten her eller over skal komme endringane for convolutional layer
    def forward(self, x:Tensor):
        
        #x = F.conv2d(x, self.conv1_filter, bias=self.conv1.bias, stride=1, padding=1)
        x= self.conv1(x)
        x = F.relu(x) 
        
        x = F.max_pool2d(x,2)
        x = self.conv2(x)
        
        x = F.relu(x)
        x = F.max_pool2d(x,2)
        x = self.conv3(x)
        
        x = F.relu(x)
        x = F.max_pool2d(x,3)
        x = torch.flatten(x, 1)
        
        x = self.fc1(x)
        
        x = F.relu(x)
        x = self.fc2(x)

        return x

summere modellen sine parametera

In [16]:
from torchsummary import summary

#Her måp det endres sidan den aksepterer ikek parametere gitt
model = ConvModel(0).to(device)
summary(model, (3, 50, 50))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 48, 48]             448
            Conv2d-2           [-1, 64, 22, 22]           9,280
            Conv2d-3            [-1, 256, 9, 9]         147,712
            Linear-4                  [-1, 128]         295,040
            Linear-5                    [-1, 2]             258
Total params: 452,738
Trainable params: 452,738
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.03
Forward/backward pass size (MB): 0.68
Params size (MB): 1.73
Estimated Total Size (MB): 2.43
----------------------------------------------------------------


Trene modellen vår her, muligens implementere wandb her for å deploye modellen og se progressjonen og sammenligne med andre modeller samnt data agumentering kan komme med i denne delen av koden?

In [17]:
def train(model, train_loader, test_loader, optimizer, criterion, n_epochs):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(n_epochs):
        # Train
        model.train()
        train_loss = 0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device).float(), labels.to(device).long()
            optimizer.zero_grad()
            outputs = model(inputs.permute(0, 3, 1, 2)).float()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_acc = 100 * correct / total
        train_loss /= len(train_loader)

        # Test
        model.eval()
        test_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device).float(), labels.to(device).long()
                outputs = model(inputs.permute(0, 3, 1, 2)).float()
                loss = criterion(outputs, labels)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        test_acc = 100 * correct / total
        test_loss /= len(test_loader)

        # Print results
        print(f"Epoch {epoch+1}/{n_epochs}")
        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%")
        print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%")

Gjør kall på modellen

In [18]:
model = ConvModel(dropout=0.5)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

train(model, trainLoader, testLoader, optimizer, criterion, n_epochs=10)

Epoch 1/10
Train Loss: 0.6935, Train Accuracy: 75.22%
Test Loss: 0.4564, Test Accuracy: 79.03%
Epoch 2/10
Train Loss: 0.3640, Train Accuracy: 84.38%
Test Loss: 0.3249, Test Accuracy: 85.97%
Epoch 3/10
Train Loss: 0.3017, Train Accuracy: 87.44%
Test Loss: 0.3110, Test Accuracy: 86.60%
Epoch 4/10
Train Loss: 0.2670, Train Accuracy: 89.07%
Test Loss: 0.2963, Test Accuracy: 87.64%
Epoch 5/10
Train Loss: 0.2505, Train Accuracy: 89.81%
Test Loss: 0.2965, Test Accuracy: 87.73%
Epoch 6/10
Train Loss: 0.2258, Train Accuracy: 90.76%
Test Loss: 0.2984, Test Accuracy: 87.33%
Epoch 7/10
Train Loss: 0.2098, Train Accuracy: 91.42%
Test Loss: 0.3484, Test Accuracy: 86.67%
Epoch 8/10
Train Loss: 0.1915, Train Accuracy: 92.34%
Test Loss: 0.2909, Test Accuracy: 88.16%
Epoch 9/10
Train Loss: 0.1719, Train Accuracy: 92.88%
Test Loss: 0.3162, Test Accuracy: 87.75%
Epoch 10/10
Train Loss: 0.1561, Train Accuracy: 93.96%
Test Loss: 0.4136, Test Accuracy: 86.59%
